In [1]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from helpers import *
import time

# '''
# Calculate the Black-Scholes implied volatility.

# Parameters:	
# price (float) – the Black-Scholes option price
# S (float) – underlying asset price
# K (float) – strike price
# t (float) – time to expiration in years
# r (float) – risk-free interest rate
# flag (str) – ‘c’ or ‘p’ for call or put.
# >>> S = 100
# >>> K = 100
# >>> sigma = .2
# >>> r = .01
# >>> flag = 'c'
# >>> t = .5
# >>> price = black_scholes(flag, S, K, t, r, sigma)
# >>> iv = implied_volatility(price, S, K, t, r, flag)
# '''

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

# Allocate Smartly Checking

In [2]:
portfolio_value = 50000

as_lst = ['DBC','IEF','IWM','QQQ','SPY','VGK','VNQ']
as_wgt = [0.028, 0.333, 0.028, 0.028, 0.528, 0.028, 0.028]

as_df = pd.DataFrame({'Allocation': as_wgt}, index = as_lst)
curr_spot = curr_batch_quotes(as_lst)
as_df = pd.concat([as_df,curr_spot[['price']]], axis = 1)

as_df['Market Value'] = portfolio_value*as_df.Allocation
as_df['Holdings'] = np.round(as_df['Market Value']/as_df.price,0)

as_df

,Allocation,price,Market Value,Holdings
symbol,,,,
DBC,0.028,17.190,1400.0,81.0
IEF,0.333,101.770,16650.0,164.0
IWM,0.028,167.450,1400.0,8.0
QQQ,0.028,181.070,1400.0,8.0
SPY,0.528,284.680,26400.0,93.0
VGK,0.028,56.970,1400.0,25.0
VNQ,0.028,82.965,1400.0,17.0


# Earnings Checking

In [49]:
# start_time = time.time()

# earnings = pd.read_csv('earnings.csv', index_col = 0)[['Earnings Date','Implied Move']]
# tick_lst = earnings.index.tolist()
# vols = current_volatility(tick_lst, roll = 22)
# earnings_df = earnings.drop_duplicates().join(vols, how='right').drop_duplicates()
# earnings_df['Lower Bound'] = earnings_df['close']*(1 - earnings_df['Implied Move']/100)
# earnings_df['Upper Bound'] = earnings_df['close']*(1 + earnings_df['Implied Move']/100)
# print("--- %s seconds ---" % (time.time() - start_time))
# earnings_df.index = earnings_df.Tickers

earnings_df

,Earnings Date,Implied Move,daily_ann,intra_ann,ovrnt_ann,close,daily_dollar_vol,Lower Bound,Upper Bound
BIO,"Aug. 7, 2018 AC",18.10,0.154327,0.287275,0.064626,312.5100,3.066702,255.945690,369.074310
INGN,"Aug. 7, 2018 AC",22.10,0.356256,0.749508,0.083775,217.5600,4.795630,169.479240,265.640760
PXD,"Aug. 7, 2018 AC",10.25,0.253401,0.360574,0.147298,187.5100,2.970830,168.290225,206.729775
JAZZ,"Aug. 7, 2018 AC",9.60,0.178899,0.386353,0.111320,180.9100,1.990099,163.542640,198.277360
JLL,"Aug. 8, 2018 BO",7.97,0.187395,0.337194,0.071068,169.6400,1.999850,156.119692,183.160308
PEN,"Aug. 7, 2018 AC",16.20,0.350734,0.666774,0.085455,147.8000,3.271040,123.856400,171.743600
IFF,"Aug. 7, 2018 AC",6.53,0.150283,0.258095,0.067320,135.1400,1.269517,126.315358,143.964642
GWPH,"Aug. 7, 2018 AC",9.30,0.275390,0.712237,0.158769,129.3900,2.255233,117.356730,141.423270
CRL,"Aug. 8, 2018 BO",6.70,0.182804,0.317536,0.063671,128.1700,1.468696,119.582610,136.757390
PRI,"Aug. 7, 2018 AC",7.72,0.143479,0.248233,0.121032,117.7000,1.048445,108.613560,126.786440


In [62]:
start_time = time.time()

focus = ['PXD', 'DIS', 'CVS', 'CLR']
focus_vol = current_volatility(focus, roll = 22)
focus_names = focus_vol.join(earnings_df[['Implied Move', 'Earnings Date']])

focus_names['Lower Bound'] = focus_names['close']*(1 - focus_names['Implied Move']/100)
focus_names['Upper Bound'] = focus_names['close']*(1 + focus_names['Implied Move']/100)

print("--- %s seconds ---" % (time.time() - start_time))

focus_names

--- 4.430296421051025 seconds ---


,daily_ann,intra_ann,ovrnt_ann,close,daily_dollar_vol,Implied Move,Earnings Date,Lower Bound,Upper Bound
PXD,0.253246,0.360604,0.147298,187.440,2.969012,10.25,"Aug. 7, 2018 AC",168.227400,206.652600
DIS,0.139100,0.273715,0.079735,116.959,1.015918,4.93,"Aug. 7, 2018 AC",111.192921,122.725079
CVS,0.227508,0.402422,0.080963,65.600,0.932131,3.42,"Aug. 8, 2018 BO",63.356480,67.843520
CLR,0.317229,0.457976,0.201509,62.460,1.253969,4.35,"Aug. 7, 2018 AC",59.742990,65.177010


In [64]:
'''
'SFLY' 'AAXN' 'WB' 'KORS'

'''
print(earnings_df[earnings_df.index.isin(['SFLY', 'AAXN', 'WB', 'KORS'])])

past_earnings('SFLY')


        Earnings Date  Implied Move  daily_ann  intra_ann  ovrnt_ann  close  \
WB    Aug. 8, 2018 BO          7.55   0.322851   0.477913   0.249806  84.43   
SFLY  Aug. 7, 2018 AC          7.75   0.205998   0.417554   0.061425  83.89   
AAXN  Aug. 7, 2018 AC          8.75   0.374529   0.748086   0.134230  67.54   
KORS  Aug. 8, 2018 BO          6.50   0.168574   0.326220   0.149939  65.19   

      daily_dollar_vol  Lower Bound  Upper Bound  
WB            1.690061    78.055535    90.804465  
SFLY          1.083939    77.388525    90.391475  
AAXN          1.577435    61.630250    73.449750  
KORS          0.686211    60.952650    69.427350  


,open,close,SFLY
timestamp,,,
2018-05-02,90.75,94.51,0.147489
2018-01-31,62.86,68.15,0.245400
2017-10-25,46.99,44.20,-0.099620
2017-07-26,46.81,50.55,0.049265
2017-04-26,52.00,50.93,-0.010353
2017-02-02,42.31,43.70,-0.169463
2016-10-27,45.49,47.21,0.115271
2016-07-28,51.43,53.18,0.073125
2016-04-28,47.55,46.55,0.004953


In [63]:
start_time = time.time()
max_gap = 5
dte_thresh = 5
money_thresh = 0.1
delta_thresh = 0.03
minimum_prem = 0.15
condors_lst = []
putspread_lst = []
callspread_lst = []
bounds_adj = 0.00

for tick in focus_names.index.drop_duplicates().tolist():
    try:
        condors, put_spreads, call_spreads = earnings_condor(tick, max_gap, dte_thresh, money_thresh)
        condors['Ticker'] = tick
        put_spreads['Ticker'] = tick
        call_spreads['Ticker'] = tick
        condors = condors[(abs(condors['Delta']) <= delta_thresh) &
                    (condors['P Short Strike'] <= focus_names.loc[tick]['Lower Bound']*(1 - bounds_adj)) &
                    (condors['C Short Strike'] >= focus_names.loc[tick]['Upper Bound']*(1 + bounds_adj)) &
                    (condors['Premium'] >= minimum_prem)].sort_values('RiskRewardRatio', ascending = False)

        put_spreads = put_spreads[(put_spreads['Short Put Strike'] <= focus_names.loc[tick]['Lower Bound']) &
                                  (put_spreads['put Spread Premium'] >= minimum_prem)].sort_values('RiskRewardRatio', ascending = False)

        call_spreads = call_spreads[(call_spreads['Short call Strike'] >= focus_names.loc[tick]['Upper Bound']) &
                                    (call_spreads['call Spread Premium'] >= minimum_prem)].sort_values('RiskRewardRatio', ascending = False)
        
        condors_lst.append(condors)
        putspread_lst.append(put_spreads)
        callspread_lst.append(call_spreads)
    except:
        continue

earnings_pspreads = pd.concat(putspread_lst).sort_values('RiskRewardRatio', ascending = False)
earnings_cspreads = pd.concat(callspread_lst).sort_values('RiskRewardRatio', ascending = False)
earnings_condors = pd.concat(condors_lst).sort_values('RiskRewardRatio', ascending = False)

print("--- %s seconds ---" % (time.time() - start_time))

prem_thresh = 0.3
rr_thresh = 0.3

filtered_condors = earnings_condors[(earnings_condors['Premium'] >= prem_thresh) &
                                     (earnings_condors['RiskRewardRatio'] >= rr_thresh)]

filtered_pspreads = earnings_pspreads[(earnings_pspreads['put Spread Premium'] >= prem_thresh) &
                                     (earnings_pspreads['RiskRewardRatio'] >= rr_thresh)]

filtered_cspreads = earnings_cspreads[(earnings_cspreads['call Spread Premium'] >= prem_thresh) &
                                     (earnings_cspreads['RiskRewardRatio'] >= rr_thresh)]

filtered_condors

--- 30.66435933113098 seconds ---


,P Short Strike,P Long Strike,C Short Strike,C Long Strike,Premium,Maxloss,Delta,Gamma,Vega,Theta,RiskRewardRatio,Underlying Price,Ticker
3126,63.0,61.5,68.0,69.5,0.40,110.0,-0.015486,-0.113650,-0.012501,0.145734,0.36,65.6,CVS
3200,63.0,61.0,68.0,70.0,0.50,150.0,-0.016295,-0.139783,-0.015484,0.181944,0.33,65.6,CVS
3726,63.0,61.5,68.5,70.0,0.35,115.0,0.012222,-0.104428,-0.011493,0.134058,0.30,65.6,CVS


# SPX Options Checking

In [ ]:
start_time = time.time()

dte_thresh = 2
money_thresh = 0.02
tick = '^SPX'
deltas_thresh = [-0.5,-0.01]

spx_options = all_options(tick)

spx_filtered_puts = spx_options[(spx_options['DTE'] <= dte_thresh) &
                                (spx_options['Type'] == 'put') &
                                (spx_options['Delta'] >= deltas_thresh[0]) &
                                (spx_options['Delta'] <= deltas_thresh[1])].reset_index()[spx_options.columns].sort_values('Strike',
                                                                                                                ascending = False)

spreads_lst = []
for idx, row in spx_filtered_puts.iterrows():
    if idx == 0:
        break
    
    temp_chains = spx_filtered_puts[spx_filtered_puts['Strike'] < row.Strike][['Strike','Bid','Ask',
                                                                               'Delta','Gamma','Theta','Vega']]
    temp_chains['DTE'] = row.DTE
    temp_chains['Short Strike'] = row.Strike
    temp_chains['Short Bid'] = row.Bid
    temp_chains['Short Ask'] = row.Ask
    
    temp_chains['Premium'] = (temp_chains['Short Bid'] - temp_chains['Ask'])*100
    temp_chains['Total Risk'] = (temp_chains['Strike'] - temp_chains['Short Strike'])*100 + temp_chains['Premium']
    temp_chains['Delta'] = temp_chains['Delta'] - row.Delta
    temp_chains['Gamma'] = temp_chains['Gamma'] - row.Gamma
    temp_chains['Theta'] = temp_chains['Theta'] - row.Theta
    temp_chains['Vega'] = temp_chains['Vega'] - row.Vega
    
    temp_chains = temp_chains[['DTE','Short Strike','Strike','Premium','Delta','Gamma','Theta','Vega','Total Risk']]
    spreads_lst.append(temp_chains)
    
pspreads_df = pd.concat(spreads_lst, axis = 0).reset_index()[temp_chains.columns]
pspreads_df['RRRatio'] = pspreads_df.Premium/(-pspreads_df['Total Risk'])

pspreads_df = pspreads_df.sort_values('RRRatio',ascending = False)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
drop_pct = 0.2
starting_pct = 0.1
spot = spx_options['Underlying_Price'][0]

spx_filtered_puts = spx_options[(spx_options['DTE'] == dte_thresh) &
                                (spx_options['Type'] == 'put') &
                                (spx_options['Strike'] >= spot*(1 - drop_pct)) &
                                (spx_options['Strike'] < spot*(1 - starting_pct))].reset_index()[spx_options.columns].sort_values('Strike',
                                                                                                                ascending = False)

spx_filtered_puts